In [ ]:
import pybullet as p
import pybullet_data
import time, math
import numpy as np
import random
from enum import Enum
from copy import deepcopy

# Getting Started

This section explains the high level APIs of pybullet and how an simulation works in a nutshell.

### Connecting to physics client
We will first starting with initiating a connection to the pybullet physics server. This is done using the `connect` function.  We can also set option parameters of the pybullet like gravity, rendering etc.



In [ ]:
#create a physics client
physics_client = p.connect(p.GUI) #P.GUI on computers with display or p.DIRECT for non-graphical version
print("The pybullet ships some useful data: ", pybullet_data.getDataPath())
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally set the search path for data files. 
p.setGravity(0,0,-9.8) #set the vector for gravity. This tells the gravity is acting in the negative z direction

### Creating the world
If you have used GUI mode, you would now be seeing a display where axis are drawn. This shows the world is empty. In order to do simulations, we need to add objects to the world. The objects can be loaded as ".urdf", ".sdf" or ".mjcf" formats. These object/robot description files tells the simulator about visual, physical and collision properties of the object/robot to be loaded. 

In [ ]:
#First we add a plane to the environment. Each object in pybullet will be given a unique id.
planeId = p.loadURDF("plane.urdf")
print("The plane id is: ", planeId)

# Now you would be seeing a plane (blue and white grid) in the GUI window. 


Now we will add a robot to the environment. The user can specify the position and orientation in which the object is to be placed. 
- The position is in (x,y,z) coordinates. 
- The PyBullet API uses quaternions to represent orientations. Since quaternions are not very intuitive for people, there are two APIs to convert between quaternions and Euler angles ( getQuaternionFromEuler and getEulerFromQuaternion )


In [ ]:
startPos = [0,0,1]
startOrientation = p.getQuaternionFromEuler([0,0,0])
print("The start position of r2d2 is: ", startPos)
print("The start orientation r2d2 is: ", startOrientation)
robotID = p.loadURDF("r2d2.urdf",startPos, startOrientation) #Loads a r2d2 robot at the specified position and orientation
# Now you would be seeing a r2d2 robot in the GUI window.


### Perform a simulation for 1000 steps

Since we have added a robot, we can simulate the robot. 

After you run the cell below, you will find  that the simulation doesn't change the pose of the robot. In fact, the simulation just simulated idle behaviour for 1000 timesteps. 

In [ ]:
p.resetBasePositionAndOrientation(robotID, startPos, startOrientation)

for i in range(1000):
    p.stepSimulation()
    time.sleep(1./240.) #This will make the simulation run at 240Hz. 
    #You can change this value to make the simulation run faster or slower. 
    #The value 1./240. means 1/240 seconds.
endPos, endOrn = p.getBasePositionAndOrientation(robotID)
print("pos after simulation: ",endPos)
print("orn after simulation: ",endOrn)

#once you are done, disconnect the physics client
p.disconnect() 

In [ ]:
p.getConnectionInfo(physics_client)

# Pick and Place

In previous section, we learnt how pybullet works in a nutshell. Now we move our attention to simulate a simple pick place behaviour. In particular we will be using Franka Emika Panda robot to pick an cube and place it at a desired location.

### Setting Up PyBullet and the Table

In [ ]:
# disconnect any physics client running already
if p.isConnected():
    p.disconnect()

# connect to the physics server
physics_client = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS, 1)
p.setGravity(0,0,-9.8)

#Load a plane
planeId = p.loadURDF("plane.urdf", basePosition = [0,0,0])

#Load a table
tableId = p.loadURDF("table/table.urdf", basePosition = [0,0,0])
# When loaded the table will be placed at the origin. You can camera position using mouse and keyboard (use CTRL + mouse click & drag to rotate) to see the table. 

In [ ]:
# Let's understand the table object

print("num joints in table: ", p.getNumJoints(tableId)) # get the number of joints in a robot/object. 0 if the object doesn't have any joints.
print("Body info: ", p.getBodyInfo(tableId)) # get the body info of the object. It outputs name and the urdf file of the object.
print("Base position and orientation of the table: ", p.getBasePositionAndOrientation(tableId)) # get the base position and orientation of the object.
print("Base velocity of the table: ", p.getBaseVelocity(tableId)) # get the base velocity of the object.
print("Base mass of the table: ", p.getDynamicsInfo(tableId, -1)[0]) # get the base mass of the object.

In [ ]:
# Get the dimensions of the Table object
collision_box = p.getAABB(tableId, -1) # get the axis aligned bounding box of the object.
print("Collision box of the table: ", collision_box)

This will be a surprise. The visual table is not the same as the collision box. The collision box only covers the top surface of the table and not the legs. This is because the way the collision is defined in urdf file of the table.  If you read the table.urdf file, it tells you that the table is made up of 5 visual objects. The first one is the top surface and the other four are the legs. The collision is defined as a box only for the top surface

```
    <collision>
      <origin rpy="0 0 0" xyz="0 0 0.6"/>
      <geometry>
	 	<box size="1.5 1 0.05"/>
      </geometry>
    </collision>
```

So, in that case, how do we get the visual data? p.getVisualShapeData() gives us the info. It return a list where each element in the list represent a visual shape. each visual shape is again a list which tells us the following in order (objectUniqueId, linkIndex, VisualGeometryType, dimensions, meshAssetFileName, localVisualFramePosition, localVisualFrameOrientation, rgbaColor, textureUniqueId) 


In [ ]:
# get visual shape data of table
table_visual_data = p.getVisualShapeData(tableId, -1)
print("Num visual shapes of the table: ", len(table_visual_data))

print("Visual shape data of the top surface: ", table_visual_data[0]) #The top is present at (0,0,0.6) with dimensions (1.5,1,0.05). 
                                                                    # This is why the collision box is from (-0.75, -0.5, 0.575) to (0.75, 0.5, 0.625) with 0.001 tolerance-limit.

print("Visual shape data of the first legs: ", table_visual_data[1]) #The first leg is present at (-0.65,0.4,0.29) with dimensions (0.1, 0.1, 0.58)

### Adding Panda Robot and Objects

In [ ]:

#Lets add a panda robot on top of the table
panda_position = [0,0.5,0.625]
panda_orientation = p.getQuaternionFromEuler([0, 0, -math.pi/2])
pandaId = p.loadURDF("franka_panda/panda.urdf", panda_position, panda_orientation, useFixedBase = 1)

Set simulation parameters. The pybullet allows us to control the speed and time of the simulation. 
By default the user has to manually execute each step of the simulation using p.stepSimulation().

When real-time simulation is enabled, the pybullet uses the internal clock to automatically simulate each simulation step without the need of p.stepSimulation(). However, the user has to be careful in sequencing the timing of actions. For example, the user has to wait untill the robot finsh moving before executing the grasp. If executed at the same time, the fingers will close while the robot is moving

In [ ]:
NUM_STEPS_PER_SECOND = 240
TIMESTEP = 1./NUM_STEPS_PER_SECOND
USE_REAL_TIME = 0
p.setTimeStep(TIMESTEP)
p.setRealTimeSimulation(USE_REAL_TIME)

In [ ]:
#lets understand panda robot more
PANDA_NUM_LINKS = p.getNumJoints(pandaId)
print("num joints in panda: ", PANDA_NUM_LINKS)

#This tells us that the panda urdf file defines 12 links. This might seem unusual because we know that the panda robot is a 7 DOF robot.
# The reason is that the urdf file defines some fixed joints that are not actuated just for the purpose of computing forward/inverse kinematics.
JointTypes = Enum("JointTypes", ["JOINT_REVOLUTE", "JOINT_PRISMATIC", "JOINT_SPHERICAL", "JOINT_PLANAR", "JOINT_FIXED"], start= 0)

#Get the joint info of the panda robot
for i in range(PANDA_NUM_LINKS):
    joints_info = p.getJointInfo(pandaId, i)
    jname = joints_info[1].decode("utf-8")
    jtype = joints_info[2]
    print(f"linkIdx: {i}, \t Joint name: {jname}, \t Joint type: {JointTypes(jtype).name}, \t joint limits: {joints_info[8:10]}")

In [ ]:
# The first 7 joints are revolute joints and the two joints representing finger is prismatic joint. Other joints are fixed joints.
PANDA_NUM_DOF = 7
PANDA_END_EFFECTOR_INDEX = 11 #Some people might use 8 as the end effector index.
PANDA_REVOLUTE_JOINTS = [0,1,2,3,4,5,6]
PANDA_PRISMATIC_JOINTS = [9,10]
PANDA_FIXED_JOINTS = [7,8,11]

PANDA_HOME_POSITION = [0, -0.785, 0, -2.356, 0, 1.571, 0.785, 0.04] #joint angles in radians for the panda home position. These values can be obtained from the Moveit ROS or Franka Control Inteface.
joint_lower_limts = [-2.8973, -1.7628, -2.8973, -3.0718, -2.8973, -0.0175, -2.8973] #joint lower limits in radians. Can be found in https://frankaemika.github.io/docs/control_parameters.html
joint_upper_limits = [2.8973, 1.7628, 2.8973, -0.0698, 2.8973, 3.7525, 2.8973] #joint upper limits in radians
joint_ranges = jr = [7]*PANDA_NUM_DOF #joint ranges for null space (@TODO: set them to proper range)

PANDA_FINGER_OPEN = 0.04
PANDA_FINGER_CLOSED = 0

#Note that the two finger may not move independent. Those are coupled.
#create a constraint to keep the fingers centered
finger_constraint = p.createConstraint(
			pandaId, 9, pandaId, 10,
			jointType=p.JOINT_GEAR,
			jointAxis=[1, 0, 0],
			parentFramePosition=[0, 0, 0],
			childFramePosition=[0, 0, 0])

p.changeConstraint(finger_constraint, gearRatio=-1, erp=0.1, maxForce=50) #These values are taken from the pybullet examples https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_robots/panda/panda_sim_grasp.py


In [ ]:
'''
Bring panda to its home position
Now we make the robot move to the given joint positions. 
The function p.setJointMotorControl2 is used to control the robot joints. You can control using POSITION_CONTROL, VELOCITY_CONTROL, TORQUE_CONTROL, etc.
Here we are using POSITION_CONTROL to control the robot joints. In particular, we are setting the target position of each joints to the given joint_positions.
'''
def is_panda_moving():
    #Panda is moving if any of the joint velocities is not close to zero
    for j in PANDA_REVOLUTE_JOINTS:
        state = p.getJointState(pandaId, j)
        this_joint_velocity = state[1]
        if not math.isclose(this_joint_velocity, 0, abs_tol=1e-5):
            return True
    return False


def panda_execute_action(max_steps = 5000):
    #manually simulate the each step of the simulation to make the robot move to the home position
    #we define a function is_panda_moving to check if panda is in a moving state. 
    #If the panda is moving, we keep simulating the environment. Once the panda stops moving, we stop the simulation.  
    p.stepSimulation() #Initiate the first simulation step
    step, status = 0, True
    while is_panda_moving() and step < max_steps:
        p.stepSimulation()
        time.sleep(TIMESTEP)
        step+=1
        if step == 2400:
            status = False
    return status

def panda_move_to_home_position(open_fingers:bool = True):
    index =0 
    for j in range(p.getNumJoints(pandaId)):
        p.changeDynamics(pandaId, j, linearDamping=0, angularDamping=0)
        info = p.getJointInfo(pandaId, j)
        jointName = info[1]
        jointType = info[2]
        if jointType == p.JOINT_REVOLUTE:           
            p.setJointMotorControl2(bodyIndex = pandaId, jointIndex=index, controlMode = p.POSITION_CONTROL, targetPosition = PANDA_HOME_POSITION[index], maxVelocity = 0.6)
            index=index+1

    #Open the gripper
    if open_fingers:
        for i in PANDA_PRISMATIC_JOINTS:
            p.setJointMotorControl2(pandaId, i, p.POSITION_CONTROL, targetPosition=PANDA_FINGER_OPEN, force=100)
    status = panda_execute_action()
    if not status:
        print("Warning: Action panda_move_to_home_position is taking long time.")
    
            
panda_move_to_home_position()
    

In [ ]:
#Lets define helper functions to move the panda robot to a given joint position depending on the concepts learnt so far.

def panda_move_to_position(joint_positions:list, include_fingers:bool = False):
    if include_fingers:
        assert len(joint_positions) == PANDA_NUM_DOF + 2, f"The joint_positions should be of length {PANDA_NUM_DOF + 2} if include_fingers is True"
    else:
        assert len(joint_positions) == PANDA_NUM_DOF, f"The joint_positions should be of length {PANDA_NUM_DOF} if include_fingers is False"
    
    index = 0
    for j in range(p.getNumJoints(pandaId)):
        info = p.getJointInfo(pandaId, j)
        jointType = info[2]
        if jointType == p.JOINT_REVOLUTE:
            p.setJointMotorControl2(bodyIndex = pandaId, jointIndex=index, controlMode = p.POSITION_CONTROL, targetPosition = joint_positions[index], maxVelocity = 0.6 )
            index=index+1
    status = panda_execute_action()
    if include_fingers:
        p.setJointMotorControl2(pandaId, 9, p.POSITION_CONTROL, targetPosition=joint_positions[-2], force=100)
        p.setJointMotorControl2(pandaId, 10, p.POSITION_CONTROL, targetPosition=joint_positions[-1], force=100) 
    if not status:
        print("Warning: Action panda_move_to_position is taking long time.")

def panda_fingers_open():
    p.setJointMotorControl2(pandaId, 9, p.POSITION_CONTROL, targetPosition=PANDA_FINGER_OPEN, force=100)
    p.setJointMotorControl2(pandaId, 10, p.POSITION_CONTROL, targetPosition=PANDA_FINGER_OPEN, force=100)
    panda_execute_action(max_steps = 50)

def panda_fingers_close():
    p.setJointMotorControl2(pandaId, 9, p.POSITION_CONTROL, targetPosition=PANDA_FINGER_CLOSED, force=100)
    p.setJointMotorControl2(pandaId, 10, p.POSITION_CONTROL, targetPosition=PANDA_FINGER_CLOSED, force=100)
    panda_execute_action(max_steps = 50)

def simulate_world(num_steps):
    for _ in range(num_steps):
        p.stepSimulation()
        time.sleep(TIMESTEP)

In [ ]:
#Lets us add a cube on the table
cube_position = [0,0,0.63]
cube_orientation = p.getQuaternionFromEuler([random.uniform(-math.pi/2, math.pi/2), random.uniform(-math.pi/2, math.pi/2), random.uniform(-math.pi/2, math.pi/2)]) #random cube orientation
cubeId = p.loadURDF("cube_small.urdf", cube_position, cube_orientation)

#If you see the cube is inside the table especially if the realtime simulation is off. When simulated, the cube will automatically come out of the table.
#Hence, we simulate the environment for 1 sec to let the cube come out of the table.
if not USE_REAL_TIME:
    simulate_world(num_steps = NUM_STEPS_PER_SECOND)

### Picking up the cube

In [ ]:
def panda_pick_object(position, orientation = None):
    assert orientation is None, "The orientation is not implemented yet."
    #restposes for null space
    rp = PANDA_HOME_POSITION

    #We will use the inverse kinematics to move the panda end effector to the cube position.
    #The function p.calculateInverseKinematics is used to calculate the inverse kinematics. The function returns the joint positions which will move the panda end effector to the cube position.
    joint_positions = p.calculateInverseKinematics(pandaId, PANDA_END_EFFECTOR_INDEX, targetPosition=position, lowerLimits=joint_lower_limts, upperLimits=joint_upper_limits, 
                                                jointRanges=joint_ranges, restPoses=rp, maxNumIterations=10000, residualThreshold=1e-4)

    # Note that we just use targetPosition. We can also optionaly give targetOrientation. 
    # However, the targetOrientation is not same as the cube orientation because we have arbitrarity rotated the cube.

    #Move to home position, move to the joint_positions found by inverse kinematics and close the fingers to grasp the object
    panda_move_to_home_position(open_fingers=True)
    panda_move_to_position(joint_positions, include_fingers=True)
    panda_fingers_close()
    panda_move_to_home_position(open_fingers=False)



In [ ]:

# First we get the position and orientation of the cube
cube_position, cube_orientation = p.getBasePositionAndOrientation(cubeId)
print("Cube position: ", cube_position)
print("Cube orientation: ", cube_orientation)

# Lets move the panda End effector to the cube position and pick object
panda_pick_object(cube_position)

### Place the object at desired position


In [ ]:
def panda_place_at_target_position(position, orientation = None):
    assert orientation is None, "The target_cube_orientation is not implemented yet."
    
    rp = PANDA_HOME_POSITION
    #We will use the inverse kinematics to move the panda end effector to the target_cube position.
    joint_positions = p.calculateInverseKinematics(pandaId, PANDA_END_EFFECTOR_INDEX, targetPosition=position,lowerLimits=joint_lower_limts, upperLimits=joint_upper_limits, 
                                               jointRanges=joint_ranges, restPoses=rp, maxNumIterations=10000, residualThreshold=1e-4)

    panda_move_to_position(joint_positions[0:7], include_fingers=False)
    panda_fingers_open()
    panda_move_to_home_position(open_fingers=False)


#Now Let us place the cube at the desired position
target_cube_position = [-0.3, 0, 0.66]
target_cube_orientation = p.getQuaternionFromEuler([0,0,0])

panda_place_at_target_position(target_cube_position)

Note, that due to multiple reasons, the cube may not be placed at the exact position. The reasons include the inaccuracies in the inverse kinematics, the inaccuracies in the simulation, etc.
The cube may fall down or the arm may collide with the object while moving to home position. Now, we will implement a simple recovery to handle such cases.


In [ ]:
#Error Recovery
error_recovery_attempts = 3
for i in range(error_recovery_attempts):
    current_cube_position, _ = p.getBasePositionAndOrientation(cubeId)
    if np.linalg.norm(np.array(current_cube_position) - np.array(target_cube_position)) < 0.01:
        break
    else:
        print(f"Error in placing the object. Retrying count {i}...")
        panda_pick_object(current_cube_position)
        panda_place_at_target_position(target_cube_position)

current_cube_position, _ = p.getBasePositionAndOrientation(cubeId)
if np.linalg.norm(np.array(current_cube_position) - np.array(target_cube_position)) < 0.01:
    print("Object placed successfully.")
else:
    print(f"Error in placing the object even after trying to recover for {error_recovery_attempts} times.")

### Pick and place through Crane grasphing

We can see that the robot hit/collide with object while moving to home position. We can handle this by performing a crane grasp. In this method, the robot moves to a position above the object and then moves down to grasp the object and vice versa. This way, the robot doesn't collide with the object while moving to home position.


In [ ]:
#Implement crane grasping and releasing

def panda_crane_pick_object(position, orientation = None):
    assert orientation is None, "The orientation is not implemented yet."
    #restposes for null space
    rp = PANDA_HOME_POSITION

    #We will use the inverse kinematics to move the panda end effector to the cube position.
    #The function p.calculateInverseKinematics is used to calculate the inverse kinematics. The function returns the joint positions which will move the panda end effector to the cube position.
    move_up_joint_positions = p.calculateInverseKinematics(pandaId, PANDA_END_EFFECTOR_INDEX,  targetPosition=[position[0], position[1], position[2]+0.09], targetOrientation = [0, math.pi, 0], 
                                                   lowerLimits=joint_lower_limts, upperLimits=joint_upper_limits, 
                                                jointRanges=joint_ranges, restPoses=rp, maxNumIterations=10000, residualThreshold=1e-4)
    
    grasp_position = p.calculateInverseKinematics(pandaId, PANDA_END_EFFECTOR_INDEX,  targetPosition = position, targetOrientation = [0, math.pi, 0], 
                                                   lowerLimits=joint_lower_limts, upperLimits=joint_upper_limits, 
                                                jointRanges=joint_ranges, restPoses=rp, maxNumIterations=10000, residualThreshold=1e-4)

    # Note that we just use targetPosition. We can also optionaly give targetOrientation. 
    # However, the targetOrientation is not same as the cube orientation because we have arbitrarity rotated the cube.

    #Move to home position, move to the joint_positions found by inverse kinematics and close the fingers to grasp the object
    panda_move_to_home_position(open_fingers=True)
    panda_move_to_position(move_up_joint_positions, include_fingers=True)
    panda_move_to_position(grasp_position, include_fingers=True)
    panda_fingers_close()
    panda_move_to_home_position(open_fingers=False)


def panda_crane_place_at_target_position(position, orientation = None):
    assert orientation is None, "The orientation is not implemented yet."
    #restposes for null space
    rp = PANDA_HOME_POSITION

    #We will use the inverse kinematics to move the panda end effector to the cube position.
    #The function p.calculateInverseKinematics is used to calculate the inverse kinematics. The function returns the joint positions which will move the panda end effector to the cube position.
    move_up_joint_positions = p.calculateInverseKinematics(pandaId, PANDA_END_EFFECTOR_INDEX,  targetPosition=[position[0], position[1], position[2]+0.09], targetOrientation = [0, math.pi, 0], 
                                                   lowerLimits=joint_lower_limts, upperLimits=joint_upper_limits, 
                                                jointRanges=joint_ranges, restPoses=rp, maxNumIterations=10000, residualThreshold=1e-4)
    
    place_position = p.calculateInverseKinematics(pandaId, PANDA_END_EFFECTOR_INDEX,  targetPosition = position, targetOrientation = [0, math.pi, 0], 
                                                   lowerLimits=joint_lower_limts, upperLimits=joint_upper_limits, 
                                                jointRanges=joint_ranges, restPoses=rp, maxNumIterations=10000, residualThreshold=1e-4)

    # Note that we just use targetPosition. We can also optionaly give targetOrientation. 
    # However, the targetOrientation is not same as the cube orientation because we have arbitrarity rotated the cube.

    #Move to home position, move to the joint_positions found by inverse kinematics and close the fingers to grasp the object
    panda_move_to_position(move_up_joint_positions, include_fingers=True)
    panda_move_to_position(place_position, include_fingers=True)
    panda_fingers_open()
    panda_move_to_position(move_up_joint_positions, include_fingers=True)
    panda_move_to_home_position(open_fingers=False)

In [ ]:

# First we get the position and orientation of the cube
cube_position, cube_orientation = p.getBasePositionAndOrientation(cubeId)
print("Cube position: ", cube_position)
print("Cube orientation: ", cube_orientation)

# Lets move the panda End effector to the cube position and pick object
panda_crane_pick_object(cube_position)

In [ ]:
#Now Let us place the cube at the desired position
target_cube_position = [-0.3, 0, 0.66]
target_cube_orientation = p.getQuaternionFromEuler([0,0,0])

panda_crane_place_at_target_position(target_cube_position)

In [ ]:
current_cube_position, _ = p.getBasePositionAndOrientation(cubeId)
print(current_cube_position)
if np.linalg.norm(np.array(current_cube_position) - np.array(target_cube_position)) < 0.05:
    print("Object placed successfully.")
else:
    print(f"Error in placing the object.")

# Rendering Images and PointClouds

The previous section described how to use inverse kinematics and joint control to perform a simple pick and place task. In this section, we will learn how to render images and point clouds from the simulation. The pybullet allows us to render RGB image, depth and segmentation masks using a synthetic camera. The user can specify the camera parameters such as field of view, aspect ratio, near and far clipping planes, etc. First, we will look into understanding the camera parameter and set up a synthetic camera.

### Understanding Camera Parameters
The pybullet uses OpenGL to render the images. The OpenGL uses a camera model which is different from the pinhole camera model. The OpenGL camera model is defined by the following parameters:
1) **View Matrix**: The view matrix defines the position and orientation of the camera in the world coordinates. The view matrix is defined by the eye position, target position and the up vector. The eye position is the position of the camera in the world coordinates. The target position is the point where the camera is looking at. The up vector is the direction of the top of the camera.
    - **eyePosition**: The position of the camera in the world coordinates
    - **targetPosition**: The position of the target point in the world coordinates
    - **upVector**: The up vector of the camera


<p align="center">
  <img src="./assets/pybullet_camera.png" alt="Alt text" title="a title">
</p>

2) **Projection Matrix**: The projection matrix defines the camera intrinsics. Pybullet provides two types of projection matrices: perspective and orthographic. The perspective projection matrix is defined by the field of view, aspect ratio, near and far clipping planes. The orthographic projection matrix is defined by the left, right, bottom, top, near and far clipping planes.

In [13]:
#The following function shows an example of how to render the desired camera view of the panda robot.
def rgba2rgb(rgba, background=(255, 255, 255)):
    """
    Convert rgba to rgb.

    Args:
        rgba (tuple):
        background (tuple):

    Returns:
        rgb (tuple):
    """
    row, col, ch = rgba.shape
    if ch == 3:
        return rgba
    assert ch == 4, 'RGBA image has 4 channels.'

    rgb = np.zeros((row, col, 3), dtype='float32')
    r, g, b, a = rgba[:, :, 0], rgba[:, :, 1], rgba[:, :, 2], rgba[:, :, 3]
    a = np.asarray(a, dtype='float32') / 255.0
    R, G, B = background
    rgb[:, :, 0] = r * a + (1.0 - a) * R
    rgb[:, :, 1] = g * a + (1.0 - a) * G
    rgb[:, :, 2] = b * a + (1.0 - a) * B
    return np.asarray(rgb, dtype='uint8')


def get_camera_image(camera_view:str, width = 1080, height = 1080, return_cam_parameters = False):
    '''
    Input: camera_view: str: The desired camera view. It can be 'top_down', 'front_perspective' or 'eye-in-hand'
    
    '''
    if camera_view == 'top_down':
        nearVal = 0.1
        farVal = 0.5
        #Rendering Top-Down View
        view_matrix = p.computeViewMatrix(cameraEyePosition=[0,0,0.8], cameraTargetPosition=[0,0,0.64], cameraUpVector=[0,0.1,-1], ) #Top-Down view
        projection_matrix = p.computeProjectionMatrix(-0.5, .5, -0.3, 0.3, nearVal, farVal)

    elif camera_view == 'front_perspective':
        #Rendering Front Perspective View
        nearVal = 0.1
        farVal = 3
        view_matrix = p.computeViewMatrix(cameraEyePosition=[0.9,0,1.2], cameraTargetPosition=[0,0,0.64], cameraUpVector=[0,0,1], )
        projection_matrix = p.computeProjectionMatrix(-0.08, 0.08, -0.1, 0.1, nearVal, farVal)

    elif camera_view == 'eye-in-hand':
        nearVal = 0.03
        farVal = 2
        #Rendering Eye-in-Hand View
        hand_position, hand_orientation, _ , _ , _ , _  = p.getLinkState(pandaId, 8, computeForwardKinematics=True)
        rot_matrix = p.getMatrixFromQuaternion(hand_orientation)
        rot_matrix = np.array(rot_matrix).reshape(3,3)
        
        #The camera is placed at the end effector of the panda robot. we find the camera-vector which defines the direction in which the camera is looking.
        init_camera_vector = np.array([0,0,1]) #z-axis
        camera_vector = np.dot(rot_matrix, init_camera_vector)

        #Note up vector is orthogonal to the camera vector. We can choose y-axix as the init_camera_vector is z-axis.
        init_up_vector = np.array([0,1,0]) #y-axis
        up_vector = np.dot(rot_matrix, init_up_vector)
        
        cam_target_pos = np.array(hand_position) + np.array([0,0,-0.02]) #[0,0,0.2] is added to the hand position to move the camera slightly below the hand.
        
        view_matrix = p.computeViewMatrix(cameraEyePosition= cam_target_pos, cameraTargetPosition=cam_target_pos+0.1*camera_vector, cameraUpVector = up_vector)
        
        #Adjust FOV and aspect if you want the gripper to be visible
        projection_matrix = p.computeProjectionMatrixFOV(fov=90, aspect=1, nearVal=nearVal, farVal=farVal) #FOV is used to set the field of view of the camera so that the view looks natural. We could use computeProjectionMatrix also but it may look unnatural. 

    #get the camera image
    _, _, rgbaImg, depthImg, segImg = p.getCameraImage(width=width, height=height, viewMatrix=view_matrix, projectionMatrix=projection_matrix, lightDirection = [0,0,1], shadow = 0, renderer=p.ER_TINY_RENDERER)

    #convert rgba to rgb
    rgbImg = rgba2rgb(rgbaImg)
    
    #convert depthBuffer to depth image using near and far values. See https://ksimek.github.io/2013/06/03/calibrated_cameras_in_opengl/ and https://stackoverflow.com/questions/70955660/how-to-get-depth-images-from-the-camera-in-pybullet 
    depthImg = farVal * nearVal / (farVal - (farVal - nearVal) * depthImg)
    
    if return_cam_parameters:
        return rgbaImg, depthImg, segImg, projection_matrix, view_matrix
    
    return rgbImg, depthImg, segImg

### Eye-in-hand camera 

In [ ]:
#display the image, depth and mask
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
def visualize(rgbImg, depthImg, segImg):
    plt.figure(figsize=(15,5))
    plt.subplot(131)
    plt.imshow(rgbImg)
    plt.title("Camera Image")
    plt.axis("off")
    plt.subplot(132)
    plt.imshow(depthImg)
    plt.title("Depth Image")
    plt.axis("off")
    plt.subplot(133)
    plt.imshow(segImg)
    plt.title("Mask Image")
    plt.axis("off")
    plt.show()


panda_move_to_home_position(open_fingers=False)
target_position = p.getBasePositionAndOrientation(cubeId)[0]
joint_positions = p.calculateInverseKinematics(pandaId, PANDA_END_EFFECTOR_INDEX, targetPosition=target_position, lowerLimits=joint_lower_limts, upperLimits=joint_upper_limits, jointRanges=joint_ranges, 
                                               restPoses=PANDA_HOME_POSITION, maxNumIterations=10000, residualThreshold=1e-4)

for i in range(7):
    p.setJointMotorControl2(pandaId, i, p.POSITION_CONTROL, targetPosition=joint_positions[i], maxVelocity=1)

p.stepSimulation()
index = 0
while is_panda_moving():
    p.stepSimulation()
    if index % 10 == 0:
        rgbImg, depthImg, segImg = get_camera_image('eye-in-hand')
    index += 1
    # visualize(rgbImg, depthImg, segImg)
    time.sleep(TIMESTEP)
    
#Do you observe the cube disappers when the end effector reaches the cube position. 
# This is because the synthetic camera is placed in end-effector enters into the cube as the end-effector reaches the cube. 
# You can modify the camera parameters and experiment with it.


### Point Clouds
The openGL camera model that is provided in pybullet outputs a non-linear depth buffer. We can convert the depth buffer into normal depth in camera frame using the following formula:
$$ depthImg = farVal * nearVal / (farVal - (farVal - nearVal) * depthImg) $$

Once, we RGB and Depth image, we can convert the depth image into point cloud as follows:

1. Get the camera intrinsics from projection matrix. Let the intrinsics be $K = \begin{bmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{bmatrix}$
2. Get the camera extrinsics from view matrix.
3. For each pixel in the depth image [x,y,d], calculate the 3D point in camera frame using the following formula:
$$ Z = d $$
$$ X = (x - c_x) * d / f_x $$
$$ Y = (y - c_y) * d / f_y $$
4. If required, finally transform the 3D point from camera frame to world frame.

---

**Getting Camera Intrinsics and Extrinsics**: 

Now, we will explain in detail how to get the camera instrincs from the projection matrix and how to get the camera extrinsics from the view matrix. pybullet got two functions to construct the projection matrix: ```computeProjectionMatrixFOV(fov, aspect, near, far)``` and ```computeProjectionMatrix(left, right, bottom, top, near, far)```. Converting from FOV params to the other group is done as follows: 
```python
top    = tan(fov/2)*near
bottom = -top
right  = top*aspect
left   = -right
# aspect = width/height
```
The code below test this conversion. Now we will see how to get the camera instrincs from the projection matrix. 

Let $n=near, f=far, r=right, l=left, t=top, b=bottom$. Then, the pybullet constructs the projection matrix using the following formula:
$$ \begin{bmatrix} \frac{2n}{r-l} & 0 & \frac{r+l}{r-l} & 0 \\ 0 & \frac{2n}{t-b} & \frac{t+b}{t-b} & 0 \\ 0 & 0 & -\frac{f+n}{f-n} & -\frac{2fn}{f-n} \\ 0 & 0 & -1 & 0 \end{bmatrix} $$

So in terms of FOV (in radians) and aspect the projection matrix is
$$ \begin{bmatrix} \frac{1}{tan(fov/2)*aspect} & 0 & 0 & 0 \\ 0 & \frac{1}{tan(fov/2)} & 0 & 0 \\ 0 & 0 & -\frac{f+n}{f-n} & -\frac{2fn}{f-n} \\ 0 & 0 & -1 & 0 \end{bmatrix} $$

It is also important to note that the pybullet uses Normalized Device Coordinates (NDC) for the depth buffer. In particular the NDC is defined as follows:
$$ \begin{bmatrix} \frac{2}{width} & 0 & 0 & -frac{r+l}{r-l}\\ 0 & \frac{2}{height} & 0 & -\frac{t+b}{t-b} \\ 0 & 0 & -\frac{2}{far-near} & -\frac{far+near}{far-near} \\ 0 & 0 & 0 & 1 \end{bmatrix} $$

The projection matrix given by pybullet is equal to NDC multiplied by the perspective matrix given by $\begin{bmatrix} fx & 0 & -cx 0 \\ 0 & fy & -cy 0 \\ 0 & 0 & n+f & n*f \\ 0 & 0 & -1 & 0 \end{bmatrix}$

So, the intrinsics are $K = \begin{bmatrix} \frac{width}{2*aspect*tan(fov/2)} & 0 & \frac{width}{2} \\ 0 & \frac{height}{2*tan(fov/2)} & \frac{height}{2} \\ 0 & 0 & 1 \end{bmatrix}$

In [14]:
##Code for checking if the conversion from FOV and Aspect to left, right, bottom, top is correct.

# fov = 60
# width = 320
# height = 240
# aspect = width / height
# near = 0.2
# far = 10

# projectionMatrix = p.computeProjectionMatrixFOV(fov, aspect, near, far)
# print(projectionMatrix)

# top = np.tan(np.deg2rad(fov)/2)*near
# bottom = -top
# right = top * aspect
# left = -right

# projectionMatrix = p.computeProjectionMatrix(left, right, bottom, top, near, far)
# print(projectionMatrix)

In [14]:
# The code is adopted from stackoverflow https://stackoverflow.com/questions/60430958/understanding-the-view-and-projection-matrix-from-pybullet
# The author didn't check the correctness of the code in detail but it seems to be correct. 
# However, we will be using open3d library to create and visualize the point cloud.

def get_intrinsics(fov, aspect, width, height):
    '''
    fov: float: field of view in degrees
    aspect: float: aspect ratio of the camera
    width: int: width of the image
    height: int: height of the image
    '''
    fx = width / (2 * aspect * np.tan(np.radians(fov / 2)))
    fy = height / (2 * np.tan(np.radians(fov / 2)))
    cx = width / 2
    cy = height / 2
    return fx, fy, cx, cy

def get_instrinsics_from_projection_matrix(projection_matrix, width, height):
    '''
    projection_matrix: list: projection matrix of the camera
    width: int: width of the image
    height: int: height of the image
    '''
    fx = projection_matrix[0] * width / 2
    fy = projection_matrix[5] * height / 2
    cx = projection_matrix[2] * width / 2 + width / 2
    cy = projection_matrix[6] * height / 2 + height / 2
    return fx, fy, cx, cy

def get_extrinsics(view_matrix):
    Tc = np.array([[1,  0,  0,  0],
                   [0,  -1,  0,  0],
                   [0,  0,  -1,  0],
                   [0,  0,  0,  1]]).reshape(4,4)
    T = np.linalg.inv(view_matrix) @ Tc

    return T

def get_transformed_pointcloud(depth_image, fx, fy, cx, cy, extrinsics_matrix, max_depth=5.0):
    # Get the point cloud
    depth_image = np.array(depth_image)
    depth_image = depth_image.astype(np.float32)
    height, width = depth_image.shape
    x = np.arange(0, width).reshape(1, -1).repeat(height, axis=0)
    y = np.arange(0, height).reshape(-1, 1).repeat(width, axis=1)
    z = depth_image
    x = (x - cx) * z / fx
    y = (y - cy) * z / fy
    pointcloud = np.stack([x, y, z], axis=-1)
    pointcloud = pointcloud.reshape(-1, 3)
    
    # filter out the points with depth > max_depth
    pointcloud = pointcloud[pointcloud[:, 2] < max_depth]
    
    pointcloud = np.concatenate([pointcloud, np.ones((pointcloud.shape[0], 1))], axis=-1)
    pointcloud = np.dot(extrinsics_matrix, pointcloud.T).T
    pointcloud = pointcloud[:, :3]
    return pointcloud



In [15]:
#Helper Functions to access the camera intrinsics and extrinsics

class CameraIntrinsics(object):
    def __init__(self, fx, fy, cx, cy, width, height):
        self.fx = fx
        self.fy = fy
        self.cx = cx
        self.cy = cy
        self.width = width
        self.height = height
    
    def get_intrinsics_matrix(self):
        return np.array([[self.fx, 0, self.cx],
                         [0, self.fy, self.cy],
                         [0, 0, 1]])
    def get_fov(self):
        return 2 * np.arctan(self.width / (2 * self.fx))
    
    def get_aspect_ratio(self):
        return self.width / self.height

    def from_fov(self, fov, aspect, width, height):
        self.fx, self.fy, self.cx, self.cy = get_intrinsics(fov, aspect, width, height)
        self.width = width
        self.height = height
        return self
    
class CameraExtrinsics(object):
    def __init__(self, extrinsics_matrix = None):
        if extrinsics_matrix is not None and not isinstance(extrinsics_matrix, np.ndarray):
            extrinsics_matrix = np.array(extrinsics_matrix).reshape(4,4)
        self.extrinsics_matrix = extrinsics_matrix
    
    def get_extrinsics_matrix(self):
        return self.extrinsics_matrix

    def from_view_matrix(self, view_matrix):
        if not isinstance(view_matrix, np.ndarray):
            view_matrix = np.array(view_matrix).reshape(4,4)
        #open 3d uses the camera coordinate system where z-axis is pointing in the opposite direction of the camera.
        #Thus, we need to flip the z-axis to make it consistent with the open3d coordinate system.
        Tc = np.array([[1,  0,  0,  0],
                       [0,  -1,  0,  0],
                       [0,  0,  -1,  0],
                       [0,  0,  0,  1]]).reshape(4,4) 
        mat = np.linalg.inv(view_matrix.T) @ Tc #TODO: Check if the transpose is required
        self.extrinsics_matrix = mat
        return self

In [ ]:
#We now use open3d library to create and visualize the point cloud.
import open3d as o3d

def visualize_pointcloud(pcd):
    if isinstance(pcd, o3d.geometry.PointCloud) or isinstance(pcd, o3d.t.geometry.PointCloud):
        o3d.visualization.draw_geometries([pcd])
    elif isinstance(pcd, list):
        o3d.visualization.draw_geometries(pcd)
    

def o3d_create_pointcloud(rgbImg, depthImg, segImg, camera_intrinsics:CameraIntrinsics, camera_extrinsics:CameraExtrinsics = None):
    device = o3d.core.Device("CPU:0")
    dtype = o3d.core.float32

    intrinsic = o3d.core.Tensor([[camera_intrinsics.fx, 0, camera_intrinsics.cx],
                                [0, camera_intrinsics.fy, camera_intrinsics.cy],
                                [0, 0, 1]], dtype, device)
    
    if camera_extrinsics is None:
        extrinsic = o3d.core.Tensor(np.eye(4), dtype, device)
    else:
        extrinsic = camera_extrinsics.get_extrinsics_matrix()
        
    rgbd = o3d.t.geometry.RGBDImage(o3d.t.geometry.Image(rgbImg.astype("float32")/255), o3d.t.geometry.Image(depthImg.astype("float32")))
    
    pcd = o3d.t.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic, depth_scale = 1.0 )
    return pcd.to_legacy()


In [17]:
#Add additional objects
#Lets add a mug on the table
cylinder_position = [0.3,0,0.63]
cylinder_orientation = p.getQuaternionFromEuler([0,0,0])
mugId = p.loadURDF("urdf/mug.urdf", cylinder_position, cylinder_orientation)

#add Lego to table
banana_position = [0.3,0.4,0.63]
banana_orientation = p.getQuaternionFromEuler([0,0,0])
legoId = p.loadURDF("lego/lego.urdf", banana_position, banana_orientation, globalScaling=2)

### Merging two point clouds
Now we will render images of mug from two different camera views and merge the point clouds to get a single point cloud. We asume that the camera intrinsics and extrinsics are known and **are not noisy**.

In [18]:
Tc = np.array([[1,  0,  0,  0],
                [0,  -1,  0,  0],
                [0,  0,  -1,  0],
                [0,  0,  0,  1]]).reshape(4,4) 

#Get the camera image from top-down view
width= 1080
height = 1080
rgbImg, depthImg, segImg, projection_matrix, view_matrix = get_camera_image('top_down', width=width, height=height, return_cam_parameters=True)
rgbImg = rgba2rgb(rgbImg)

#Get the camera intrinsics and extrinsics
fx, fy, cx, cy = get_instrinsics_from_projection_matrix(projection_matrix, width, height)
camera_intrinsics = CameraIntrinsics(fx, fy, cx, cy, width, height)

#generate the point cloud for the mug
mug_mask = segImg == 4
mug_img = rgbImg.copy() * mug_mask.reshape(height, width, 1).repeat(3, axis=2)
mug_depth = depthImg.copy() *mug_mask
mug_pcd = o3d_create_pointcloud(mug_img, mug_depth, mug_mask, camera_intrinsics)

#convert the point cloud from camera coordinate system to world coordinate system
mug_pcd_t = deepcopy(mug_pcd).transform(np.linalg.inv(np.array(view_matrix).reshape(4,4)).T @ Tc)

In [19]:
#Get the camera image from front perspective view
width= 1080
height = 1080
rgbImg_front, depthImg_front, segImg_front, projection_matrix_front, view_matrix_front = get_camera_image('front_perspective', width=width, height=height, return_cam_parameters=True)
rgbImg_front = rgba2rgb(rgbImg_front)

#Get the camera intrinsics and extrinsics
fx, fy, cx, cy = get_instrinsics_from_projection_matrix(projection_matrix_front, width, height)
camera_intrinsics_front = CameraIntrinsics(fx, fy, cx, cy, width, height)

mug_mask_front = segImg_front == 4
mug_img_front = rgbImg_front.copy() * mug_mask_front.reshape(height, width, 1).repeat(3, axis=2)
mug_depth_front = depthImg_front.copy() * mug_mask_front
mug_pcd_front = o3d_create_pointcloud(mug_img_front, mug_depth_front, mug_mask_front, camera_intrinsics_front)

#convert the point cloud from camera coordinate system to world coordinate system
mug_pcd_front_t = deepcopy(mug_pcd_front).transform(np.linalg.inv(np.array(view_matrix_front).reshape(4,4)).T @ Tc)

In [ ]:
visualize_pointcloud([mug_pcd_t, mug_pcd_front_t])

# Learning to Grasp

In [ ]:

# Add the specified directory to the pybullet search path
p.setAdditionalSearchPath("/home/coding-cheetah/Research/simulator/envs/bullet/assets")

# Load the object from the specified directory
object_position = [-0.3, 0.7, 0.63]
object_orientation = p.getQuaternionFromEuler([0, 0, 0])

# Load the texture
texture_id = p.loadTexture("ycb/001_chips_can/tsdf/textured.png")

# Create a visual shape for the object with texture
visual_shape_id = p.createVisualShape(
    shapeType=p.GEOM_MESH,
    fileName="ycb/001_chips_can/tsdf/textured.obj",
    meshScale=[1, 1, 1],
    textureUniqueId=texture_id
)

# Create a multi-body for the object
object_id = p.createMultiBody(
    baseMass=1,
    baseInertialFramePosition=[0, 0, 0],
    baseVisualShapeIndex=visual_shape_id,
    basePosition=object_position,
    baseOrientation=object_orientation
)

# Print the object ID to confirm it has been loaded
print("The object ID is: ", object_id)


In [ ]:
for i in range(1000):
    p.stepSimulation()
    time.sleep(1/240)

In [ ]:
p.setAdditionalSearchPath("/home/coding-cheetah/Research/simulator/envs/bullet/assets")
p.loadURDF("ycb/001_chips_can/tsdf/chips_can.urdf", [0, 0.2, 0.63], p.getQuaternionFromEuler([0, 0, 0]))

In [ ]:
p.setAdditionalSearchPath("/home/coding-cheetah/Research/simulator/envs/bullet/assets")
objects = p.loadSDF("gso/5_HTP/model.sdf", globalScaling = 3)
position = [0.2, 0.3, 0.64]
orientation = p.getQuaternionFromEuler([0, 0, 0])
p.resetBasePositionAndOrientation(objects[0], position, orientation)

In [ ]:
p.setAdditionalSearchPath("/home/coding-cheetah/Research/simulator/envs/bullet/assets")
object = p.loadSDF('ycb/011_banana/google_512k/model.sdf' )
position = [0.3, 0.3, 0.63]
orientation = p.getQuaternionFromEuler([0, 0, 0])
p.resetBasePositionAndOrientation(object[0], position, orientation)
# texture_id = p.loadTexture("ycb/024_bowl/google_512k/texture_map.png")
# p.changeVisualShape(object[0], -1, textureUniqueId=texture_id)

In [ ]:
light_position = [0, 0, 1.5]  # Change this to position the light source
p.addUserDebugLine([0, 0, 0], light_position, [1, 1, 1], lineWidth=2)

In [ ]:
object

In [ ]:
p.addUserDebugLine([0, 0, 0], [5, 5, 5], [1, 1, 1], lineWidth=2)
p.addUserDebugLine([0, 0, 0], [-5, -5, 5], [1, 1, 1], lineWidth=2)

In [ ]:
p.setAdditionalSearchPath("/home/coding-cheetah/Research/simulator/envs/bullet/assets")
# Load the reference object using SDF
reference_objects = p.loadSDF("ycb/024_bowl/google_512k/model.sdf")
reference_object = reference_objects[0]  # Assuming only one object in the SDF

# Set initial position and orientation for the reference object
ref_position = [0, 0, 0.64]  # Reference object at the origin
ref_orientation = p.getQuaternionFromEuler([0, 0, 0])
p.resetBasePositionAndOrientation(reference_object, ref_position, ref_orientation)

In [ ]:


# 2. Get the bounding box of the reference object
aabb_min, aabb_max = p.getAABB(reference_object)
ref_dim = np.array(aabb_max) - np.array(aabb_min)  # Dimensions of the reference object
ref_center = np.array([(aabb_min[i] + aabb_max[i]) / 2 for i in range(3)])  # Center position

# Calculate the left region based on dimensions
left_region_min = [aabb_min[0] - ref_dim[0], aabb_min[1], aabb_min[2]]
left_region_max = [aabb_min[0], aabb_max[1], aabb_max[2]]

# 3. Sample a position and orientation in the left region
def sample_position_and_orientation(region_min, region_max):
    sampled_position = [
        np.random.uniform(region_min[0], region_max[0]),
        np.random.uniform(region_min[1], region_max[1]),
        np.random.uniform(region_min[2], region_max[2]),
    ]
    sampled_orientation = p.getQuaternionFromEuler(
        [np.random.uniform(-np.pi, np.pi) for _ in range(3)]
    )
    return sampled_position, sampled_orientation

# 4. Check for collision
def is_colliding(object_id, position, orientation):
    # Temporarily move the object to check for collisions
    temp_objs = p.loadSDF("ycb/011_banana/google_512k/model.sdf")
    temp_obj = temp_objs[0]
    p.resetBasePositionAndOrientation(temp_obj, position, orientation)
    for _ in range(1000):
        p.stepSimulation()
        time.sleep(1/240)
    contact_points = p.getClosestPoints(temp_obj, reference_object, distance=0.0)
    p.removeBody(temp_obj)
    return len(contact_points) > 0

# 5. Place the target object
target_placed = False
for _ in range(100):  # Try 100 samples
    target_position, target_orientation = sample_position_and_orientation(left_region_min, left_region_max)
    if not is_colliding(reference_object, target_position, target_orientation):
        # Successfully found a valid position
        target_objs = p.loadSDF("ycb/011_banana/google_512k/model.sdf")
        p.resetBasePositionAndOrientation(target_objs[0], target_position, target_orientation)
        target_placed = True
        print("Target object placed at:", target_position)
        break

if not target_placed:
    print("Failed to place the target object without collision.")

